In [23]:
import pyodbc
import numpy as np
import os
import pandas as pd
import scipy.signal as signal
import math as math
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import threading as thd
import time
import warnings

In [18]:
anterior = load_model('C:/Users/GPU/Desktop/Model/model_final/model_anterior_2.h5')
antero_lateral = load_model('C:/Users/GPU/Desktop/Model/model_final/model_antero-lateral_2.h5')
antero_septal = load_model('C:/Users/GPU/Desktop/Model/model_final/model_antero-septal_2.h5')
inferior = load_model('C:/Users/GPU/Desktop/Model/model_final/model_inferior_2.h5')
infero_lateral = load_model('C:/Users/GPU/Desktop/Model/model_final/model_infero-lateral_2.h5')

In [3]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=120.124.84.221;DATABASE=TestECG;UID=GPU;PWD=sql')
cursor = cnxn.cursor()
print(cursor)

In [4]:
#cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=120.124.84.221;DATABASE=TestECG;UID=GPU;PWD=sql')
cnxn_220 = pyodbc.connect('DRIVER={SQL Server};SERVER=120.124.84.220;DATABASE=TestECG;UID=sa;PWD=sql')
cursor_220  = cnxn_220 .cursor()
print(cursor_220)

In [39]:
#hos_id = '0000000001'
hos_id = '0000000003'

#ID = 'james20718'

In [24]:
def Analysis_Data(hos_id):
    warnings.filterwarnings("ignore")
    cursor.execute("select TOP 60000 i,ii,iii,avr,avl,avf,v1,v2,v3,v4,v5,v6 from patient order by seq ASC")
    row_2 = cursor.fetchall()
    
    cursor.execute(" select TOP 60000 i,ii,iii,avr,avl,avf,v1,v2,v3,v4,v5,v6 from patient2 where hos_id= "+hos_id+"and status IS NULL and status_accept IS NULL order by seq2 ASC")
    row = cursor.fetchall()
    
    cursor.execute("select distinct ID , Unique_Value from patient2 where hos_id= '"+hos_id+"'and status IS NULL and status_accept IS NULL") 
    ID = cursor.fetchall()
    ID = np.asarray(ID) #List 轉換
    #print(ID)
    if ID != []:
        ID_1 = ID[0][0]
        print(ID)
        s = ID[0][1]
        #print(s)
    else :
        print('please check id ')
        return
    row3 = []
    row3.append(row)
    row3.append(row_2)

    
    all_ecg = []


    for i in range(0,2):
        Detect_ecg = np.asarray(row3[i])
        Detect_ecg = Detect_ecg.astype(str).astype(float)
        #all_ecg = []
        if(len(Detect_ecg)!=60000):   
            print("----Part1 -not d60000 fail")
            #return
        else:
            Detect_ecg=Detect_ecg[:].transpose()
        t_ecg = []
        for num in range(0,len(Detect_ecg)):
            fs = 250
            ecg = np.ravel(Detect_ecg[num])

            buffer_plot =[]

            state = 0 
            buffer_base=[]
            window = round(fs/25)

            ecg = ecg[:]
            ecg_raw =ecg
            f1=0.5
            f2=45
            Wn=np.array([f1,f2])*1/fs
            N = 3
            a,b = signal.butter(N,Wn, 'bandpass') 
            ecg = signal.filtfilt(a,b,ecg)
            ecg=signal.resample(ecg, 2000, t=None, axis=0, window=None)
            t_ecg.append(ecg)

        all_ecg.append(np.ravel(t_ecg).T)

        test_Data = np.asarray(all_ecg)
    #print(test_Data.shape,'-----')

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_test_Data = scaler.fit_transform(test_Data)

    reshape_scaled_test_Data =scaled_test_Data.reshape(scaled_test_Data.shape[0],24000,1).astype('float32')
    #print(reshape_scaled_test_Data)

    anterior_pb = anterior.predict_proba(reshape_scaled_test_Data,verbose=0)
    antero_lateral_pb = antero_lateral.predict_proba(reshape_scaled_test_Data,verbose=0)
    antero_septal_pb = antero_septal.predict_proba(reshape_scaled_test_Data,verbose=0)
    inferior_pb = inferior.predict_proba(reshape_scaled_test_Data,verbose=0)
    infero_lateral_pb = infero_lateral.predict_proba(reshape_scaled_test_Data,verbose=0)

    #anterior_pb_c = anterior.predict_classes(reshape_scaled_test_Data,verbose=0)
    #antero_lateral_pb_c = antero_lateral.predict_classes(reshape_scaled_test_Data,verbose=0)
    #antero_septal_pb_c = antero_septal.predict_classes(reshape_scaled_test_Data,verbose=0)
    #inferior_pb_c = inferior.predict_classes(reshape_scaled_test_Data,verbose=0)
    #infero_lateral_pb_c = infero_lateral.predict_classes(reshape_scaled_test_Data,verbose=0)
    #model_total = anterior_pb_c[0]+antero_lateral_pb_c[0]+antero_septal_pb_c[0]+inferior_pb_c[0]+infero_lateral_pb_c[0]
    #print(model_total)
    # Sick
    #total_Sick = (anterior_pb[0][0]+antero_lateral_pb[0][0]+antero_septal_pb[0][0]+inferior_pb[0][0]+infero_lateral_pb[0][0])
    anterior_percent= ("%.2f" %((anterior_pb[0][0])*100))
    antero_lateral_percent= ("%.2f" %((antero_lateral_pb[0][0])*100))
    antero_septal_percent= ("%.2f" %((antero_septal_pb[0][0])*100))
    inferior_percent= ("%.2f" %((inferior_pb[0][0])*100))
    infero_lateral_percent= ("%.2f" %((infero_lateral_pb[0][0])*100))



    Analysis_Data = [anterior_percent,
            antero_lateral_percent,
            antero_septal_percent,
            inferior_percent,
            infero_lateral_percent]
    #print(Analysis_Data)
    Analysis_Data_Covert = np.array(Analysis_Data)

    try:
        if((anterior_pb[0][0]>anterior_pb[0][1])or(antero_lateral_pb[0][0]>antero_lateral_pb[0][1])or(antero_septal_pb[0][0]>antero_septal_pb[0][1])or(inferior_pb[0][0]>inferior_pb[0][1])or(infero_lateral_pb[0][0]>infero_lateral_pb[0][1])): 
        #if((5-model_total)<=3):
            cursor.execute("UPDATE  TOP(60000)  patient2 SET status = '0',status_accept = '0' Where hos_id= "+hos_id+" and status IS NULL ").commit()
            status = '0'
            #cursor.execute("INSERT INTO Analysis_results(hos_id,anterior_pb,antero_lateral_pb,antero_septal_pb,inferior_pb,infero_lateral_pb,status,status_accept)"+ 
                           #"VALUES('"+hos_id+"','"+anterior_percent+"','"+antero_lateral_percent+"','"+antero_septal_percent+"','"+inferior_percent+"','"+infero_lateral_percent+"','"+status+"','"+status+"')").commit()
           
            print("Positive");
            #------
            #少一寫一個 條件 Analysis_results 如果is null 要初始值 
            #------
        
            cursor_220.execute("INSERT INTO Analysis_results(Unique_Value, hos_id,ID,anterior_pb,antero_lateral_pb,antero_septal_pb,inferior_pb,infero_lateral_pb,status,status_accept)"+ 
                           "VALUES('"+s+"','"+hos_id+"','"+ID_1+"','"+anterior_percent+"','"+antero_lateral_percent+"','"+antero_septal_percent+"','"+inferior_percent+"','"+infero_lateral_percent+"','"+status+"','"+status+"')").commit()
    except NameError as Error:
                cursor.rollback()

    try:
        #if((5-model_total)>3):
        if((anterior_pb[0][0]<anterior_pb[0][1])and(antero_lateral_pb[0][0]<antero_lateral_pb[0][1])and(antero_septal_pb[0][0]<antero_septal_pb[0][1])and(inferior_pb[0][0]<inferior_pb[0][1])and(infero_lateral_pb[0][0]<infero_lateral_pb[0][1])):
            cursor.execute("UPDATE  TOP(60000)  patient2 SET status = '1', status_accept = '5' Where hos_id= "+hos_id+" and status IS NULL ").commit()
            status ='1'

           #cursor.execute("INSERT INTO Analysis_results(hos_id,anterior_pb,antero_lateral_pb,antero_septal_pb,inferior_pb,infero_lateral_pb,status,status_accept)"+ 
                           #"VALUES('"+hos_id+"','"+anterior_percent+"','"+antero_lateral_percent+"','"+antero_septal_percent+"','"+inferior_percent+"','"+infero_lateral_percent+"','"+status+"','"+status+"')").commit()
            print("Negative");
            cursor_220.execute("INSERT INTO Analysis_results(Unique_Value,hos_id,ID,anterior_pb,antero_lateral_pb,antero_septal_pb,inferior_pb,infero_lateral_pb,status,status_accept)"+ 
                           "VALUES('"+s+"','"+hos_id+"','"+ID_1+"','"+anterior_percent+"','"+antero_lateral_percent+"','"+antero_septal_percent+"','"+inferior_percent+"','"+infero_lateral_percent+"','"+status+"','"+status+"')").commit()
    except NameError as Error:
                cursor.rollback()


    return Analysis_Data_Covert
    
    

In [40]:
ans= Analysis_Data(hos_id)
print(ans)

[['Cindy12345' '0000000002']]
Negative
['0.01' '0.04' '0.31' '0.00' '0.00']


In [ ]:
#Health Care Data Center 

In [14]:
#def fn():
 #   a = Analysis_Data(hos_id)
  #  print(a)
   # thd.Timer(10,fn).start()
#fn()